In [5]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.7 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 1.7 MB/s eta 0:00:00
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import ssl
import gensim
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ssl._create_default_https_context = ssl._create_unverified_context


class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss
        

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch}')
        self.epoch += 1

        

# TF-IDF и Suggest

Этот семинар будет посвящен работе с текстами. Мы будем решать задачу о которой говорили на лекции: будем реализовывать suggest.

### План семинара


* [Задача suggest: многоклассовая классификация](#Задача-suggest)
* [Обучаем модель линейной регрессии](#Обучаем-модель-лог-регрессии)
    - Делаем бейзлайн
    - Улучшаем бейзлайн
    
* Обучаем word2vec
* Вычисляем признаки заголовков
* Смотрим метрику в нашей задаче

**Работа на семинаре**
* Реализуем tf-idf 

**Домашнее задание**
* Реализовать класс сервиса suggest-by-title

### Задача suggest

Подача без suggest:

<img src="https://ucarecdn.com/05ab935a-ff65-4d75-afcb-6eb0d71c5d44/" width="700">

Подача с suggest:
<img src="https://ucarecdn.com/5e1684f1-eec5-4054-9e43-08be8d9acbcb/" width="700">

Задача заключается в том, чтобы определить к какой категории относится объявление используя заголовок. Категорий в выборке 54, задача, соответственно многоклассовая классификация.

Задача многоклассовой классификации методом one-versus-all:

<img src="https://ucarecdn.com/2abe5812-3f6e-42cf-97a4-a35a6de58c6d/" width="534">

### Смотрим на данные

In [2]:
data_train = pd.read_csv('suggest_train.csv', index_col=0)
data_test = pd.read_csv('suggest_test.csv', index_col=0)
w2v_train = pd.read_csv('unlabeled_data.csv', index_col=0)

In [3]:
data_train.head()

,title,category_id,name
item_id,,,
335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...
434016,Линолеум комитекс лин печора орфей 431,15,Для дома и дачи|Ремонт и строительство|Стройма...
237890,Распродажа имущества предприятия. Компьютеры,7,Бытовая электроника|Настольные компьютеры


In [26]:
data_train.category_id.value_counts()

37    447
23    433
27    421
35    420
15    419
12    417
39    417
11    409
53    407
22    405
45    404
18    402
29    402
21    400
10    395
44    391
50    390
47    388
26    387
24    383
2     382
28    376
17    375
36    370
49    368
6     368
19    361
31    361
0     360
34    360
43    359
52    358
30    355
46    354
8     354
20    353
16    351
41    350
9     350
32    346
7     346
3     345
48    343
13    343
25    342
51    341
42    341
4     341
14    334
33    324
38    321
40    311
5     311
1     309
Name: category_id, dtype: int64

In [27]:
data_train.category_id.nunique()

54

In [28]:
sorted(data_train.category_id.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53]

In [31]:
data_train['name'].apply(lambda x: len(x.split('|'))).describe()

count    20000.00000
mean         3.06075
std          0.44460
min          2.00000
25%          3.00000
50%          3.00000
75%          3.00000
max          4.00000
Name: name, dtype: float64

### Обучаем модель линейной регрессии

#### Разбиваем данные на трейн и валидацию

Идеи:
- Частые слова зашумляют выборку;
- Редкие слова дают модели переобучиться;
- n-граммы позволят частично учитывать порядок;
- Регуляризация позволит уменьшить эффект переобучения;
- ???

In [32]:
train_titles, val_titles, y_train, y_val = train_test_split(
    data_train['title'],
    data_train['category_id'],
    random_state=1
)

In [170]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.4 MB/s eta 0:00:0000:0100:010m
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [181]:
import re

In [171]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/buchkovv/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [252]:
# ["для", "продам", "на", "из"]
# verb_pattern = r'^(?!.*ть$)static.+'
# verb_pattern = r'\b\w\w+(?!ть)\b'
verb_pattern = r'\b\w\w+[^(ть)]\b'

count_vectorizer = CountVectorizer(stop_words=russian_stopwords, token_pattern=verb_pattern)
count_vectorizer

CountVectorizer(stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                token_pattern='\\b\\w\\w+[^(ть)]\\b')

In [194]:
# count_vectorizer.get_feature_names_out()

In [140]:
# train_titles.unique

In [253]:
X_train = count_vectorizer.fit_transform(train_titles.values)
X_val = count_vectorizer.transform(val_titles.values)

In [254]:
feature_names = count_vectorizer.get_feature_names_out()

In [124]:
# count = X_train.toarray().sum(axis=0)

In [255]:
dictionary = dict(zip(feature_names, count))
dict_sorted = {f: c for f, c in sorted(dictionary.items(), key=lambda x: x[1], reverse=True)}

In [256]:
print(dict_sorted)

{'бухари': 1153, 'минима': 931, 'кованная ': 514, 'newton ': 386, 'pauline ': 353, 'front': 317, 'выступления': 287, 'лошадь-': 250, 'контейнерами ': 235, 'dvb-': 220, 'social ': 217, 'пиломатериалы ': 206, 'law': 198, 'роберт ': 194, 'минимум': 185, 'ролл': 172, 'кенгурятник ': 163, 'деревянных ': 145, 'кованный': 140, 'конного ': 140, 'контейнера': 140, 'брелок': 138, 'поднос ': 136, 'блюдцами': 133, 'истории ': 133, 'пижамы': 128, 'с1005341': 127, 'бастет ': 125, '065b8209 ': 123, 'душевный ': 123, '44.': 118, 'брюшную ': 113, 'люминесцентный ': 105, 'платья ': 103, '90x21mm ': 102, 'джинсовые ': 100, 'haibo ': 99, 'диам': 99, 'подростка ': 97, '14x2.': 93, 'петра ': 93, 'династия ': 92, 'нюська': 92, 'dynamic': 90, 'isopanel ': 89, 'брелок-': 87, 'кор': 87, 'wizard ': 85, 'кронштейн ': 85, 'hm-': 84, 'metallica': 84, 'дерева': 84, 'праздники': 84, 'detomaso ': 83, 'marine-': 83, 'serie ': 83, 'mexx': 82, 'пепельница ': 82, 'игра': 79, 'поясная': 78, 'блузки': 77, 'watt': 76, 'немец

In [117]:
# unique, counts = np.unique(feature_names, return_counts=True)

In [138]:
# print(np.asarray((unique, counts)).T)

In [257]:
print(len(count_vectorizer.vocabulary_))
count_vectorizer.vocabulary_

20079


{'бильярдный ': 9067,
 'стол': 17799,
 'крассула ': 12673,
 'оватта': 14611,
 'денежное ': 10611,
 'дерево': 10628,
 'фиксатор': 18843,
 'вальгус ': 9449,
 'подставка ': 15570,
 'для ': 10807,
 'цветов': 19213,
 'комплект ': 12361,
 'мебели': 13578,
 'ибица': 11446,
 'wi-': 8016,
 'fi ': 4152,
 'роутер ': 16672,
 'link ': 5319,
 'dir-': 3646,
 '615': 1886,
 'электрический ': 19811,
 'самовар': 16870,
 'sony ': 7165,
 'experia ': 4051,
 'snell ': 7128,
 'farcry ': 4107,
 'ps4': 6476,
 'продам': 16017,
 'смартфон ': 17337,
 'nokia ': 6026,
 'lumia ': 5413,
 '630 ': 1903,
 'dual ': 3760,
 'чёрный': 19435,
 'minelab ': 5709,
 '17 ': 521,
 'smart ': 7104,
 'ctx ': 3448,
 '3030': 1221,
 'ведьмак ': 9509,
 'дикая ': 10747,
 'охота': 14910,
 'продаются ': 16024,
 'коньки ': 12447,
 'фигурные ': 18835,
 'женские': 11124,
 'ретро ': 16545,
 'свеча ': 17000,
 'очень ': 14931,
 'красивая': 12642,
 'настенная ': 14240,
 'полка': 15659,
 'навесная': 14124,
 'пиджак ': 15257,
 'из ': 11499,
 'экокожи

In [249]:
count_vectorizer.vocabulary_['стол']

17838

In [48]:
# X_train[0, 15273]
# 0.739
# 0.7402

# 0.7400
# 0.6936
# 0.6946

In [258]:
np.random.seed(1)
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs')
log_reg.fit(X_train, y_train)

LogisticRegression()

In [259]:
preds = log_reg.predict(X_val)
accuracy_score(preds, y_val)

0.6946

### Учим w2v

Идеи:
- Обучить качественные векторные представление слов на неразмеченных данных;
- Получить из них вектора заголовков;
- Оценить адекватность;
- Оценить качество решения нашей задачи;
- Большая размерность позволит получить более качественные векторные представления;
- Сравнить CBOW и skip-gram;
- ???

In [70]:
import re
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import SGDClassifier

# From CountVectorizer() (default pattern)
WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('unlabeled_data.csv',  index_col=0)
print(w2v_train_data.shape)
w2v_train_data.head()

(200000, 2)


,item_id,title
30000,377939,1 рубль царские 10 штук
30001,384779,"Напольный насос giyo высокого давления, с мано..."
30002,276889,Sony Xperia m2 Dual
30003,155718,Zoom B2.1U+ AC Процессор эффектов для бас-гитары
30004,260705,Люстра Favourite petite 1574-7P


In [72]:
w2v_train_data.values[0, 1]

'1 рубль царские 10 штук'

In [73]:
sentences = [reg_exp.findall(s.lower()) for s in w2v_train_data.title]
sentences[:3]

[['рубль', 'царские', '10', 'штук'],
 ['напольный', 'насос', 'giyo', 'высокого', 'давления', 'манометр'],
 ['sony', 'xperia', 'm2', 'dual']]

In [74]:
w2v_model = Word2Vec(sg=1,)
w2v_model.build_vocab(sentences)
w2v_model.train(
    sentences,
    total_examples=w2v_model.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 1437889.75
Loss after epoch 1: 869712.25
Loss after epoch 2: 722732.75
Loss after epoch 3: 643189.25
Loss after epoch 4: 585492.0
Loss after epoch 5: 499774.5
Loss after epoch 6: 477723.5
Loss after epoch 7: 460667.5
Loss after epoch 8: 444513.0
Loss after epoch 9: 424619.5
Loss after epoch 10: 421782.5
Loss after epoch 11: 412589.5
Loss after epoch 12: 405083.5
Loss after epoch 13: 398461.5
Loss after epoch 14: 366084.0
Loss after epoch 15: 339888.0
Loss after epoch 16: 334103.0
Loss after epoch 17: 330086.0
Loss after epoch 18: 326449.0
Loss after epoch 19: 325956.0


(11628398, 14684740)

In [75]:
w2v_model.wv.similar_by_word('продам')

[('продаю', 0.8961564898490906),
 ('породам', 0.7989452481269836),
 ('новую', 0.686275839805603),
 ('продадим', 0.616396963596344),
 ('отдам', 0.5739603042602539),
 ('кабину', 0.5723707675933838),
 ('хорошую', 0.5699202418327332),
 ('подам', 0.5693252086639404),
 ('женскую', 0.5608842968940735),
 ('норковую', 0.5580729842185974)]

In [76]:
w2v_model.wv.similar_by_word('айфон')

[('iphone', 0.8230351805686951),
 ('iphone5', 0.8207362294197083),
 ('ipone', 0.8072177171707153),
 ('6plus', 0.7946138978004456),
 ('iphone4', 0.794045090675354),
 ('айфоны', 0.7828352451324463),
 ('айфон4', 0.7741789221763611),
 ('айфонов', 0.7714698910713196),
 ('5c', 0.768720805644989),
 ('5с', 0.7647431492805481)]

In [77]:
class Word2VecTransformer:
    
    def __init__(self, w2v_model, word_pattern):
        
        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):
            
            title_vector = np.zeros((self.w2v_model.wv.vector_size,))
            tokens = self.re.findall(title.lower())
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    title_vector += self.w2v_model.wv.get_vector(token)
                    
            X_transformed[i] = title_vector
        
        return X_transformed

In [78]:
w2v_transformer = Word2VecTransformer(w2v_model=w2v_model, word_pattern=WORD_PATTERN)

train_w2v = w2v_transformer.transform(train_titles.values)
val_w2v = w2v_transformer.transform(val_titles.values)

In [79]:
log_reg = SGDClassifier()
log_reg.fit(train_w2v, y_train)

SGDClassifier()

In [80]:
preds = log_reg.predict(val_w2v)
accuracy_score(preds, y_val)

0.721

### Работа на семинаре: [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class TFIDFVectorizer(BaseEstimator, TransformerMixin):
    
    def __init__():
        pass
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        pass

### Работа на семинаре: suggest на подаче

* Реализовать [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) transformer
* Обучить линейную модель на признаках TF-IDf сравнить с реализацией из sklearn
    - Разобраться с каждым гиперпараметром реализации TF-IDf из sklearn
    
* Обучить word2vec на данных data/unlabeled_data.csv
* Попробовать улучшить результат который получили с помощью w2v

### Домашнее задание

Теоритическая часть (обязательно).

1. Посмотреть видео на [3BlueBrowb](https://youtu.be/aircAruvnKk)

Практическая часть.

**Реализовать класс Suggester, который возвращает от 1 до 5 наиболее вероятных категорий по введённой строке.**

accuracy по top1 > 0.78

In [ ]:
from typing import List

class Suggester:
    
    def __init__(self, max_suggest_count=5, default_suggest=''):
        self.max_suggest_count = max_suggest_count
        self.default_suggest = default_suggest
        
    def suggest(self, title: str):
        print('Для дома и дачи|Посуда и товары для кухни|Посуда')
        print('Хобби и отдых|Коллекционирование|Другое')
    
    def test_suggest(self):
        
        title = input()
        while title != 'stop':
            
            suggest = self.suggest(title)

            if suggest:
                print(suggest)
            else:
                print(self.default_suggest)
            
            title = input()

In [ ]:
sug = Suggester()
sug.test_suggest()